We load the packages.

In [126]:
import numpy as np
import matplotlib.pyplot as plt

Then, one defines the domain here. I think the easiest way to do this is to hardcode the domain once, then refine it.

1. square
2. triangle
3. L-shaped

I used a domain of size 1 in x, that I can easily scale be dividing by the number of elements in x (L) of size h such that L\*h=1 always. However there are (L+1)^2 nodes here for a square. There are therefore L\*H elements in this construction.

Then, the array coordinates is of shape ((L+1)\*(H+1), 2), where each row is (x-position, y-position). I try to go anticlockwise for everynodes.

In [127]:
#number of elements in x and y
L=2
H=2
coordinates1=np.zeros(((L+1)*(H+1),2))
i=0
while True:
    coordinates1[i,0]=(i%(L+1))/L
    coordinates1[i,1]=i//(L+1)/L
    i+=1
    if i==(L+1)*(H+1):
        break
print(coordinates1)

[[0.  0. ]
 [0.5 0. ]
 [1.  0. ]
 [0.  0.5]
 [0.5 0.5]
 [1.  0.5]
 [0.  1. ]
 [0.5 1. ]
 [1.  1. ]]


Now, it is time for the elements. So every element is defined counterclockwise. The array elements3 is of shape (2\*L\*H,3), where each row is like this (node1, node2, node3).

In [128]:
"""
elements3=np.zeros((2*L*H,4))
i=0
while True:
    elements3[i,0]=i
    if (i-(i//(L)))%(L)==0:
        elements3[i,1]=i-(i//(L+1))
    if (i-(i//(L)))%(L)!=0:
        elements3[i,1]=0
    elements3[i,2]=(i+1)%2+i%2*(L+1)
    elements3[i,3]=1
    i+=1
    if i==L*H*2:
        break
"""
elements3=np.array([[0,1,3],[1,4,3],[1,2,4],[2,5,4],[3,4,6],[4,7,6],[4,5,7],[5,8,7]])
print(elements3)

[[0 1 3]
 [1 4 3]
 [1 2 4]
 [2 5 4]
 [3 4 6]
 [4 7 6]
 [4 5 7]
 [5 8 7]]


Suppose now that the given square is such that on the sides it is the Neumann BC and on the top and bottom the Dirichlet BC. Both are size (2\*H,2) and (2\*L,2) respectively.

In [159]:
"these are triangle 0, 3, 4, 7 resp." 
neumannBC=np.array([[0,3],[0,0],[0,0],[2,5],[3,6],[0,0],[0,0],[5,8]])
"these are triangle 0, 2, 5, 7 resp."
dirichletBC=np.array([[0,1],[0,0],[1,2],[0,0],[0,0],[6,7],[0,0],[7,8]])

We can now implement the basis function, eta:

In [153]:
def eta(i,j,x,y):
    "where i is the index of the triangle in 0 to 7, j is the index of the function in 0 to 2 and (x,y) are the\
    coordinates of the point within the triangle"
    n=np.linalg.det([[1,x,y],\
                               [1,coordinates1[elements3[i,(j+1)%3],0],coordinates1[elements3[i,(j+1)%3],1]],\
                               [1,coordinates1[elements3[i,(j+2)%3],0],coordinates1[elements3[i,(j+2)%3],1]]])/\
                np.linalg.det([[1,coordinates1[elements3[i,(j)%3],0],coordinates1[elements3[i,(j)%3],1]],\
                               [1,coordinates1[elements3[i,(j+1)%3],0],coordinates1[elements3[i,(j+1)%3],1]],\
                               [1,coordinates1[elements3[i,(j+2)%3],0],coordinates1[elements3[i,(j+2)%3],1]]])
    return n

def area(i):
    "where i is the index of the triangle in 0 to 7"
    T=(1/2)*np.linalg.det([[coordinates1[elements3[i,1],0]-coordinates1[elements3[i,0],0],\
                            coordinates1[elements3[i,2],0]-coordinates1[elements3[i,0],0]],\
                           [coordinates1[elements3[i,1],1]-coordinates1[elements3[i,0],1],\
                            coordinates1[elements3[i,2],1]-coordinates1[elements3[i,0],1]]])
    return T

def Deta(i,j,x,y):
    "where i is the index of the triangle in 0 to 7, j is the index of the function in 0 to 2 and (x,y) are the\
    coordinates of the point within the triangle"
    dn=(1/2/area(i))*np.array([coordinates1[elements3[i,(j+1)%3],1]-coordinates1[elements3[i,(j+2)%3],1],\
                 coordinates1[elements3[i,(j+2)%3],0]-coordinates1[elements3[i,(j+1)%3],0]])
    return dn

And now we can do the stiffness matrix:

In [154]:
def Smat(i):
    "where i is the number of the triangle between 0 and 7"
    G=np.dot(np.linalg.inv(np.array([[1,1,1],\
                [coordinates1[elements3[i,0],0],coordinates1[elements3[i,1],0],coordinates1[elements3[i,2],0]],\
                [coordinates1[elements3[i,0],1],coordinates1[elements3[i,1],1],coordinates1[elements3[i,2],1]]])),\
             np.array([[0,0],[1,0],[0,1]]))
    M=area(i)/2*np.dot(G,G.T)
    return M

def Smat2(i,j,k):
    M=area(i)*np.dot(Deta(i,j,coordinates1[elements3[i,(j)%3],0],coordinates1[elements3[i,(j)%3],1]),\
                     Deta(i,k,coordinates1[elements3[i,(k)%3],0],coordinates1[elements3[i,(k)%3],1]).T)
    return M
print(Smat(0),"\n",np.array([[Smat2(1,0,0),Smat2(1,1,0),Smat2(1,2,0)],\
                      [Smat2(1,0,1),Smat2(1,1,1),Smat2(1,2,1)],\
                      [Smat2(1,0,2),Smat2(1,1,2),Smat2(1,2,2)]]))

[[ 0.5  -0.25 -0.25]
 [-0.25  0.25  0.  ]
 [-0.25  0.    0.25]] 
 [[ 0.5 -0.5  0. ]
 [-0.5  1.  -0.5]
 [ 0.  -0.5  0.5]]


Now, we can deal with the f part of the equation; assume that f=1 everywhere, for starters. (the code still works for an f that is not a constant, since I made the dependance on each triangle's centroid explicit) We do similarly with the Neumann boundary condition, and with Dirichlet BC

In [155]:
def f(x,y):
    "Insert here the proper function that one wants"
    return np.sin(x)*np.sin(y)

def g(x,y):
    "Insert here the proper function that one wants"
    return x*y

def ud(x,y):
    "insert here the proper dirichlet bc"
    return 0

def force(i):
    centerX=1/3*(coordinates1[elements3[i,0],0]+coordinates1[elements3[i,1],0]+coordinates1[elements3[i,2],0])
    centerY=1/3*(coordinates1[elements3[i,0],1]+coordinates1[elements3[i,1],1]+coordinates1[elements3[i,2],1])
    intf=1/6*np.linalg.det([[coordinates1[elements3[i,1],0]-coordinates1[elements3[i,0],0],\
                          coordinates1[elements3[i,2],0]-coordinates1[elements3[i,0],0]],\
                         [coordinates1[elements3[i,1],1]-coordinates1[elements3[i,0],1],\
                          coordinates1[elements3[i,2],1]-coordinates1[elements3[i,0],1]]])*f(centerX,centerY)
    return intf

def neumann(i):
    "where i is the triangle index ranging from 0 to 3 representing triangles (0, 4, 3, 7), note that not all triangles have an edge with a neumann BC"
    centerX=(coordinates1[neumannBC[i,1],0]+coordinates1[neumannBC[i,0],0])/2
    centerY=(coordinates1[neumannBC[i,1],1]+coordinates1[neumannBC[i,0],1])/2
    intg=(coordinates1[neumannBC[i,1],1]-coordinates1[neumannBC[i,0],1])/2*g(centerX,centerY)
    return intg

Now we have to define the RHS and the LHS in order to compute the Dirichlet BC:

In [172]:
"some basic initialzation"
A=np.zeros((len(elements3),3,3))
b=np.zeros(len(elements3))
u=np.zeros(len(elements3))

for i in np.arange(len(elements3)):
    A[i]=Smat(i)
    "dirichlet BC"
    if dirichletBC[i,1]-dirichletBC[i,0]>0:
        u[i]=ud(coordinates[dirichletBC[i,0],coordinates[dirichletBC[i,1]]
    b[i]=force(i)+neumann(i)-np.dot(A[i],u[i])
    "solution computation"
    u[i]=np.dot(np.linalg.inv(A[i]),b[i])
    

SyntaxError: invalid syntax (<ipython-input-172-27432415f8af>, line 11)

I know that what I did doesn't work, I realised it 5 minutes before the due time... I mixed up two indices and that is why it doesn't work...